In [33]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [39]:
import re
from datetime import datetime
import pandas as pd

from echolalia.parser import WhatsAppParser
from echolalia.contextualizer import Contextualizer

In [63]:
parser = WhatsAppParser()

S3_BUCKET_NAME = "smcphers-echolalia"
CHAT_LOG_FILENAME = "data/_chat.txt"

# Parse the chat log into dicts and then into a DataFrame
messages = parser.parse_chat_log(bucket=S3_BUCKET_NAME, chat_log_filename=CHAT_LOG_FILENAME)

In [64]:
messages

,timestamp,user,message,exception,chatline,time_diff
0,2022-01-17 23:15:56,Cat,‎Messages and calls are end-to-end encrypted. ...,None,"[1/17/22, 11:15:56 PM] Cat: ‎Messages and call...",NaT
1,2022-01-17 23:15:56,Selwyn-Lloyd McPherson,Hello?!,None,"[1/17/22, 11:15:56 PM] Selwyn-Lloyd McPherson:...",0 days 00:00:00
2,2022-01-17 23:16:26,Cat,HI,None,"[1/17/22, 11:16:26 PM] Cat: HI",0 days 00:00:30
3,2022-01-17 23:17:24,Cat,oh good i do have it on my computer already,None,"[1/17/22, 11:17:24 PM] Cat: oh good i do have ...",0 days 00:00:58
4,2022-01-17 23:17:24,Selwyn-Lloyd McPherson,"Success!\n‎[1/17/22, 11:19:47 PM] Selwyn-Lloyd...",None,"[1/17/22, 11:17:24 PM] Selwyn-Lloyd McPherson:...",0 days 00:00:00
...,...,...,...,...,...,...
53904,2024-08-06 12:44:43,Cat,Cough cough wheezeeeee cough,None,"[8/6/24, 12:44:43 PM] Cat: Cough cough wheezee...",1 days 19:38:08
53905,2024-08-08 13:13:14,Cat,My turn in the er,None,"[8/8/24, 1:13:14 PM] Cat: My turn in the er",2 days 00:28:31
53906,2024-08-08 16:41:41,Cat,Ya girls got cirrhosis,None,"[8/8/24, 4:41:41 PM] Cat: Ya girls got cirrhosis",0 days 03:28:27
53907,2024-08-08 16:41:49,Cat,Among other things,None,"[8/8/24, 4:41:49 PM] Cat: Among other things",0 days 00:00:08


In [70]:
# Create a "group" whenever the user changes
messages['group'] = (messages['user'] != messages['user'].shift()).cumsum()

In [71]:
messages

,timestamp,user,message,exception,chatline,time_diff,group
0,2022-01-17 23:15:56,Cat,‎Messages and calls are end-to-end encrypted. ...,None,"[1/17/22, 11:15:56 PM] Cat: ‎Messages and call...",NaT,1
1,2022-01-17 23:15:56,Selwyn-Lloyd McPherson,Hello?!,None,"[1/17/22, 11:15:56 PM] Selwyn-Lloyd McPherson:...",0 days 00:00:00,2
2,2022-01-17 23:16:26,Cat,HI,None,"[1/17/22, 11:16:26 PM] Cat: HI",0 days 00:00:30,3
3,2022-01-17 23:17:24,Cat,oh good i do have it on my computer already,None,"[1/17/22, 11:17:24 PM] Cat: oh good i do have ...",0 days 00:00:58,3
4,2022-01-17 23:17:24,Selwyn-Lloyd McPherson,"Success!\n‎[1/17/22, 11:19:47 PM] Selwyn-Lloyd...",None,"[1/17/22, 11:17:24 PM] Selwyn-Lloyd McPherson:...",0 days 00:00:00,4
...,...,...,...,...,...,...,...
53904,2024-08-06 12:44:43,Cat,Cough cough wheezeeeee cough,None,"[8/6/24, 12:44:43 PM] Cat: Cough cough wheezee...",1 days 19:38:08,27953
53905,2024-08-08 13:13:14,Cat,My turn in the er,None,"[8/8/24, 1:13:14 PM] Cat: My turn in the er",2 days 00:28:31,27953
53906,2024-08-08 16:41:41,Cat,Ya girls got cirrhosis,None,"[8/8/24, 4:41:41 PM] Cat: Ya girls got cirrhosis",0 days 03:28:27,27953
53907,2024-08-08 16:41:49,Cat,Among other things,None,"[8/8/24, 4:41:49 PM] Cat: Among other things",0 days 00:00:08,27953


In [72]:
 # Group by this new "group" column and concatenate the values in "messages"
messages_combined = messages.groupby('group', as_index=False).agg({
    'user': 'first',  # Take the first value of 'column1' for each group
    'message': '. '.join  # Concatenate the values of 'column2'
})

In [73]:
messages_combined

,group,user,message
0,1,Cat,‎Messages and calls are end-to-end encrypted. ...
1,2,Selwyn-Lloyd McPherson,Hello?!
2,3,Cat,HI. oh good i do have it on my computer already
3,4,Selwyn-Lloyd McPherson,"Success!\n‎[1/17/22, 11:19:47 PM] Selwyn-Lloyd..."
4,5,Cat,yes!. https://photobucket.com/u/intercat can y...
...,...,...,...
27948,27949,Cat,I have a few quarters in my desk
27949,27950,Selwyn-Lloyd McPherson,Count that shit up. Minus a few for laundry
27950,27951,Cat,"About a dollar of Uzbek money\n‎[8/1/24, 4:15:..."
27951,27952,Selwyn-Lloyd McPherson,I just called al Italia airlines to see what k...


,column1,column2,group
0,A,apple,1
1,A,banana,1
2,B,orange,2
3,C,grape,3
4,A,peach,4
5,A,plum,4
6,C,melon,5
